**IMPORTANDO BIBLIOTECAS**

In [ ]:
import pandas as pd 
import numpy as np

**OBTENDO DADOS**

In [ ]:
try:
    print('Obtendo dados...')

    ENDERECO_DADOS = 'https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv'

    df = pd.read_csv(ENDERECO_DADOS, sep=',', encoding='utf-8')[['review_title','overall_rating']]

    #excluindo dados não existentes(NaN)
    
    df = df.dropna(subset=['review_title', 'overall_rating'])

    #Transformando colunas em arrays
    texts = np.array(df['review_title'])
    rating = np.array(df['overall_rating'])

    print(df.head())

except Exception as e:
    print('Erro ao obter dados: ', e)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer #type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences #type: ignore

try:
    print('Vetorizando texto...')

    tokenizer = Tokenizer()

    tokenizer.fit_on_texts(texts)

    vetores = tokenizer.texts_to_sequences(texts)

    padded_vetores = pad_sequences(vetores)
    
    print(padded_vetores)
except Exception as e:
    print('Erro ao obter dados: ', e)

**REDE NEURAL**

In [ ]:
from tensorflow.keras.models import Sequential #type: ignore
from tensorflow.keras.layers import Embedding, LSTM, Dense #type: ignore
from tensorflow.keras.optimizers import Adam #type: ignore
try:
    print('Construindo a rede neural...')

    VOCAB_SIZE = len(tokenizer.word_index) + 1

    MAX_SEQUENCE_LENGHT = padded_vetores.shape[1]

    VETOR_LENGHT = int(np.sqrt(VOCAB_SIZE))

    model = Sequential()

    model.add(Embedding(input_dim=VOCAB_SIZE,output_dim=VETOR_LENGHT,input_length = MAX_SEQUENCE_LENGHT))

    model.add(LSTM(128))
    
    model.add(Dense(1))

    model.build(input_shape=(None,MAX_SEQUENCE_LENGHT))

    otimizador = Adam(learning_rate=0.0005)

    model.compile(optimizer=otimizador, loss='mean_squared_error')

    model.summary()
    print('Modelo configurado')


except Exception as e:
    print('Erro ao construir dados: ', e)    

In [ ]:
from sklearn.model_selection import train_test_split

try:
    print('Treinar o modelo de rede neural')

    X_train, X_test, y_train, y_test = train_test_split(padded_vetores, rating, test_size=0.2, random_state=42)

    model.fit(
        X_train,
        y_train,
        epochs=5,
        batch_size=128,
        validation_data=(X_test,y_test)
    )
except Exception as e:
    print('Erro ao treinar a rede neural: ', e)    

In [35]:
novos_textos = ['Muito bom, gostei bastante. Top demais! Compensa muito','Nâo recomendo, péssimo produto. Não funciona']

novas_sequencias = tokenizer.texts_to_sequences(novos_textos)
novas_sequencias_padded = pad_sequences(novas_sequencias)

predicoes = model.predict(novas_sequencias_padded)

print('Previsões: ' ,predicoes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Previsões:  [[ 3.3658648 ]
 [-0.01342357]]
